# Stage 4: Two-Pass Sentiment Inference

Apply two fine-tuned DistilBERT models to all classified articles:
- **Pass 1 (Labor)**: labor_only + both → labor_stance_model → labor_sentiment
- **Pass 2 (Railroad)**: railroad_only + both → railroad_outlook_model → railroad_sentiment

"Both" articles are scored on **both axes** (richest data points).

Features: **ONNX Runtime**, **keyword-centered truncation**, **checkpointing** every 10K articles.

In [ ]:
import json
import os
import re
import time
import logging
import numpy as np
import torch
from pathlib import Path
from tqdm import tqdm

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("data/inference_log.log")
    ]
)
log = logging.getLogger(__name__)

LABOR_MODEL_PATH = Path('models/labor_stance_model')
RAILROAD_MODEL_PATH = Path('models/railroad_outlook_model')
CLASSIFIED_DIR = Path('data/classified_articles')
OUTPUT_DIR = Path('data/sentiment_results')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_FILE = OUTPUT_DIR / 'all_sentiment_results.json'
CHECKPOINT_INTERVAL = 10_000

LABOR_LABEL_MAP = {0: 'pro_labor', 1: 'anti_labor', 2: 'neutral'}
RAILROAD_LABEL_MAP = {0: 'optimistic', 1: 'pessimistic', 2: 'neutral'}

In [ ]:
def load_json(filepath):
    with open(filepath, encoding='utf-8') as f:
        return json.load(f)

def save_json(filepath, data):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)

In [ ]:
# --- Keyword definitions for keyword-centered truncation ---
LABOR_KEYWORDS = {
    'labor union': 3, 'trade union': 3, 'labor strike': 3, 'labor riot': 3,
    'collective bargaining': 3, 'labor movement': 3, 'strikebreaker': 3,
    'scab labor': 3, 'working men': 3, 'workingmen': 3,
    'knights of labor': 3, 'eight hour': 3,
    'striker': 2, 'strikers': 2, 'picket': 2, 'lockout': 2,
    'boycott': 2, 'walkout': 2, 'arbitration': 2, 'picketing': 2,
    'strike': 1, 'strikes': 1, 'wage': 1, 'wages': 1,
    'workers': 1, 'laborers': 1,
}
RAILROAD_KEYWORDS = {
    'railroad company': 3, 'railroad strike': 3, 'railroad workers': 3,
    'railway company': 3, 'union pacific': 3, 'central pacific': 3,
    'northern pacific': 3, 'pennsylvania railroad': 3,
    'baltimore and ohio': 3, 'railroad line': 3,
    'locomotive': 2, 'locomotives': 2, 'brakeman': 2,
    'freight car': 2, 'passenger car': 2, 'rail road': 2,
    'railroad': 1, 'railway': 1, 'train': 1, 'trains': 1,
}

LABOR_PATTERNS = {kw: (re.compile(r'\b' + re.escape(kw) + r'\b', re.IGNORECASE), w)
                  for kw, w in LABOR_KEYWORDS.items()}
RAILROAD_PATTERNS = {kw: (re.compile(r'\b' + re.escape(kw) + r'\b', re.IGNORECASE), w)
                     for kw, w in RAILROAD_KEYWORDS.items()}


def best_keyword_position(text, axis):
    """Find char position of highest-weighted keyword for the given axis."""
    patterns = LABOR_PATTERNS if axis == 'labor' else RAILROAD_PATTERNS
    best_pos, best_weight = 0, 0
    for kw, (regex, weight) in patterns.items():
        match = regex.search(text)
        if match and weight > best_weight:
            best_weight = weight
            best_pos = match.start()
    return best_pos


print("Keyword patterns loaded.")

## Load Both Models

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(LABOR_MODEL_PATH)


def smart_truncate(text: str, axis: str, max_length: int = 512) -> str:
    """Keyword-centered truncation: keep tokens around the best keyword match."""
    tokens = tokenizer.tokenize(text)
    if len(tokens) <= max_length - 2:
        return text

    center_char = best_keyword_position(text, axis)
    center_token = int(len(tokens) * center_char / len(text)) if len(text) > 0 else 0

    window = max_length - 2
    half = window // 2
    start = max(0, center_token - half)
    end = start + window
    if end > len(tokens):
        end = len(tokens)
        start = max(0, end - window)

    return tokenizer.convert_tokens_to_string(tokens[start:end])


def load_model(model_path, model_name):
    """Load a model with ONNX if available, else PyTorch fallback."""
    use_onnx = False
    try:
        from optimum.onnxruntime import ORTModelForSequenceClassification

        onnx_path = model_path / 'model.onnx'
        if not onnx_path.exists():
            log.info(f"Converting {model_name} to ONNX format...")
            model = ORTModelForSequenceClassification.from_pretrained(
                model_path, export=True
            )
            model.save_pretrained(model_path)
            log.info("ONNX conversion complete.")
        else:
            model = ORTModelForSequenceClassification.from_pretrained(model_path)
        use_onnx = True
        print(f"{model_name}: Using ONNX Runtime")
    except ImportError:
        from transformers import AutoModelForSequenceClassification
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        model.eval()
        print(f"{model_name}: Using PyTorch (install optimum for 2-3x speedup)")

    metadata_path = model_path / 'training_metadata.json'
    if metadata_path.exists():
        metadata = load_json(metadata_path)
        print(f"  Val accuracy: {metadata.get('val_accuracy', 'N/A')}")

    return model, use_onnx


labor_model, labor_onnx = load_model(LABOR_MODEL_PATH, 'Labor Stance')
railroad_model, railroad_onnx = load_model(RAILROAD_MODEL_PATH, 'Railroad Outlook')

In [ ]:
def predict_batch(texts, model, axis, use_onnx, label_map, batch_size=32):
    """Predict sentiment for a batch of texts using keyword-centered truncation."""
    results = []

    for i in range(0, len(texts), batch_size):
        batch_texts = [smart_truncate(t, axis) for t in texts[i:i+batch_size]]

        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt',
        )

        if use_onnx:
            outputs = model(**inputs)
        else:
            with torch.no_grad():
                outputs = model(**inputs)

        logits = outputs.logits
        if isinstance(logits, torch.Tensor):
            logits_np = logits.numpy()
        else:
            logits_np = np.array(logits)

        predictions = np.argmax(logits_np, axis=-1)
        exp_logits = np.exp(logits_np - np.max(logits_np, axis=-1, keepdims=True))
        probs = exp_logits / np.sum(exp_logits, axis=-1, keepdims=True)

        for j, pred in enumerate(predictions):
            results.append({
                'sentiment': label_map[int(pred)],
                'confidence': round(float(probs[j][pred]), 3),
            })

    return results


# Quick sanity test
test_texts = [
    "The brave workers stood firm against the tyrannical factory owners in the labor strike.",
    "The riotous mob of strikers destroyed property and threatened honest citizens.",
    "Workers at the Pullman factory went on strike Tuesday morning.",
]
print("Labor model test:")
for text, r in zip(test_texts, predict_batch(test_texts, labor_model, 'labor', labor_onnx, LABOR_LABEL_MAP)):
    print(f"  [{r['sentiment']:>12}] ({r['confidence']:.2f}) {text[:70]}")

rr_texts = [
    "The new transcontinental railroad line will bring prosperity to the western territories.",
    "Another terrible railroad accident claimed twelve lives yesterday.",
    "The Pennsylvania Railroad reported quarterly earnings on Monday.",
]
print("\nRailroad model test:")
for text, r in zip(rr_texts, predict_batch(rr_texts, railroad_model, 'railroad', railroad_onnx, RAILROAD_LABEL_MAP)):
    print(f"  [{r['sentiment']:>12}] ({r['confidence']:.2f}) {text[:70]}")

## Load Classified Articles

In [ ]:
labor_only = load_json(CLASSIFIED_DIR / 'labor_only.json')
railroad_only = load_json(CLASSIFIED_DIR / 'railroad_only.json')
both = load_json(CLASSIFIED_DIR / 'both.json')

print(f"Labor-only:    {len(labor_only):,}")
print(f"Railroad-only: {len(railroad_only):,}")
print(f"Both:          {len(both):,}")

labor_pass_articles = labor_only + both
railroad_pass_articles = railroad_only + both

print(f"\nLabor pass:    {len(labor_pass_articles):,} articles")
print(f"Railroad pass: {len(railroad_pass_articles):,} articles")

## Two-Pass Inference

Each pass has its own checkpoint file for independent resumability.

In [ ]:
def run_inference_pass(articles, model, axis, use_onnx, label_map, checkpoint_path):
    """Run inference on articles with checkpointing. Returns dict: article_id -> {sentiment, confidence}."""
    results = {}
    start_idx = 0

    if checkpoint_path.exists():
        checkpoint = load_json(checkpoint_path)
        results = checkpoint['results']
        start_idx = checkpoint['processed']
        log.info(f"Resuming {axis} pass: {start_idx:,} already done")
    else:
        log.info(f"Starting {axis} pass: {len(articles):,} articles")

    CHUNK_SIZE = 1000
    BATCH_SIZE = 32
    start_time = time.time()

    for chunk_start in tqdm(range(start_idx, len(articles), CHUNK_SIZE),
                            initial=start_idx // CHUNK_SIZE,
                            total=(len(articles) + CHUNK_SIZE - 1) // CHUNK_SIZE,
                            desc=f"{axis} inference"):
        chunk = articles[chunk_start:chunk_start + CHUNK_SIZE]
        texts = [a['text'] for a in chunk]

        preds = predict_batch(texts, model, axis, use_onnx, label_map, batch_size=BATCH_SIZE)

        for article, pred in zip(chunk, preds):
            results[article['article_id']] = pred

        processed = chunk_start + len(chunk)

        if processed % CHECKPOINT_INTERVAL == 0:
            save_json(checkpoint_path, {'processed': processed, 'results': results})
            elapsed = time.time() - start_time
            rate = (processed - start_idx) / elapsed if elapsed > 0 else 0
            remaining = (len(articles) - processed) / rate if rate > 0 else 0
            log.info(f"Checkpoint ({axis}): {processed:,}/{len(articles):,} "
                     f"({rate:.0f}/sec, ~{remaining/3600:.1f}h left)")

    if checkpoint_path.exists():
        os.remove(checkpoint_path)

    elapsed = time.time() - start_time
    log.info(f"{axis} pass done: {len(results):,} articles in {elapsed/60:.1f}min")
    return results

In [ ]:
# Pass 1: Labor model on labor_only + both
print("=== PASS 1: Labor Stance ===")
labor_results = run_inference_pass(
    articles=labor_pass_articles,
    model=labor_model,
    axis='labor',
    use_onnx=labor_onnx,
    label_map=LABOR_LABEL_MAP,
    checkpoint_path=OUTPUT_DIR / 'checkpoint_labor.json',
)
print(f"Labor results: {len(labor_results):,} articles scored")

In [ ]:
# Pass 2: Railroad model on railroad_only + both
print("=== PASS 2: Railroad Outlook ===")
railroad_results = run_inference_pass(
    articles=railroad_pass_articles,
    model=railroad_model,
    axis='railroad',
    use_onnx=railroad_onnx,
    label_map=RAILROAD_LABEL_MAP,
    checkpoint_path=OUTPUT_DIR / 'checkpoint_railroad.json',
)
print(f"Railroad results: {len(railroad_results):,} articles scored")

## Merge Results

Combine both passes into a single output. "Both" articles get both columns filled.

In [ ]:
all_articles = labor_only + railroad_only + both
article_info = {a['article_id']: a for a in all_articles}

merged = []
for article_id, info in article_info.items():
    labor_pred = labor_results.get(article_id)
    rr_pred = railroad_results.get(article_id)

    entry = {
        'article_id': article_id,
        'lccn': info['lccn'],
        'issn': info['issn'],
        'year': info['year'],
        'category': info['category'],
        'labor_sentiment': labor_pred['sentiment'] if labor_pred else None,
        'labor_confidence': labor_pred['confidence'] if labor_pred else None,
        'railroad_sentiment': rr_pred['sentiment'] if rr_pred else None,
        'railroad_confidence': rr_pred['confidence'] if rr_pred else None,
    }
    merged.append(entry)

save_json(OUTPUT_FILE, merged)

print(f"Merged results: {len(merged):,} articles")
print(f"Saved to: {OUTPUT_FILE}")

has_labor = sum(1 for r in merged if r['labor_sentiment'] is not None)
has_rr = sum(1 for r in merged if r['railroad_sentiment'] is not None)
has_both = sum(1 for r in merged if r['labor_sentiment'] is not None and r['railroad_sentiment'] is not None)
print(f"\nCoverage:")
print(f"  With labor sentiment:    {has_labor:,}")
print(f"  With railroad sentiment: {has_rr:,}")
print(f"  With both sentiments:    {has_both:,} (should match 'both' article count)")

In [ ]:
import pandas as pd

df = pd.DataFrame(merged)

print("=== Labor Sentiment Distribution ===")
print(df['labor_sentiment'].value_counts(dropna=False))

print("\n=== Railroad Sentiment Distribution ===")
print(df['railroad_sentiment'].value_counts(dropna=False))

print("\n=== By Category ===")
for cat in ['labor', 'railroad', 'both']:
    subset = df[df['category'] == cat]
    print(f"\n{cat.upper()} ({len(subset):,} articles):")
    if subset['labor_sentiment'].notna().any():
        print(f"  Labor stance:     {subset['labor_sentiment'].value_counts().to_dict()}")
    if subset['railroad_sentiment'].notna().any():
        print(f"  Railroad outlook: {subset['railroad_sentiment'].value_counts().to_dict()}")

print("\n=== Average Confidence ===")
print(f"  Labor:    {df['labor_confidence'].mean():.3f}")
print(f"  Railroad: {df['railroad_confidence'].mean():.3f}")

print("\n=== Historical Validation ===")
for year, event in [(1877, 'Great Railroad Strike'), (1886, 'Haymarket Affair')]:
    subset = df[df['year'] == year]
    if len(subset) > 0:
        print(f"\n{year} ({event}): {len(subset)} articles")
        if subset['labor_sentiment'].notna().any():
            print(f"  Labor:    {subset['labor_sentiment'].value_counts().to_dict()}")
        if subset['railroad_sentiment'].notna().any():
            print(f"  Railroad: {subset['railroad_sentiment'].value_counts().to_dict()}")